In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
df = pd.read_csv("data_final_resample_standardization.csv")
df.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EMERGENCYSTATE_MODE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,TARGET
0,0.751970,0.235286,1.375052,0.403220,1.695226,-0.141886,0.860965,-0.413916,1.240573,-0.299187,...,0.949231,0.921827,0.769546,0.820445,0.003860,0.99243,0.710023,0.911945,0.918624,0
1,-0.591704,0.193017,0.366399,0.669629,0.588342,2.002004,-1.782893,2.354147,0.143480,-0.844557,...,0.949231,0.921827,0.769546,0.820445,0.234323,0.99243,0.289977,0.911945,0.918624,0
2,0.751970,-0.060600,-0.582922,-0.649225,-0.453432,-0.696247,-0.850283,-0.434673,0.067698,0.207228,...,0.949231,0.921827,0.769546,0.820445,0.221104,0.99243,0.289977,0.911945,0.918624,0
3,-0.591704,-0.145139,-0.938917,-1.204133,-0.844097,-0.370580,1.172374,-0.417441,-0.002838,-0.109034,...,0.949231,0.921827,0.230454,0.179555,0.124913,0.99243,0.710023,0.911945,0.918624,0
4,-0.591704,0.218378,3.214360,1.821318,3.713662,-1.264537,0.777275,-0.433341,0.828726,-0.933692,...,0.949231,0.921827,0.769546,0.820445,0.010952,0.99243,0.710023,0.911945,0.918624,0


In [3]:
encoder = joblib.load("save_files/encoder_dict.joblib")
dict_stand = joblib.load("save_files/dict_standardization.joblib")
model_logistic = joblib.load("save_files/logistic_classifier_model_sel.joblib") # Use the lighter model

### Data for Prediction

In [4]:
# Target = 1 (Loan Rejected)
applicant_income = 202500.0
credit_amount = 406597.5
loan_annuity = 24700.5
goods_price = 351000.0
region_pop = 0.018801
applicant_age = -9461
day = "WEDNESDAY"
hour = 10
org_type = "Business Entity Type 3"
credit_bureau = 1.0
days_employed = -637
days_reg = -3648.0
days_id = -2120
days_phone = -1134.0
exp_mode = 0.9722
area_mode = 0.0149
source_2 = 0.2629485927471776
source_3 = 0.13937578009978951

In [5]:
# Target = 0 (Loan Accepted)
applicant_income0 = 270000.0
credit_amount0 = 1293502.5
loan_annuity0 = 35698.5
goods_price0 = 1129500.0
region_pop0 = 0.003540999999999999
applicant_age0 = -16765
day0 = "MONDAY"
hour0 = 11
org_type0 = "School"
credit_bureau0 = 0
days_employed0 = -1188
days_reg0 = -1186.0
days_id0 = -291
days_phone0 = -828.0
exp_mode0 = 0.9851
area_mode0 = 0.0714
source_20 = 0.6222457752555098
source_30 = 0.7295666907060153

### 1. Categorical Data

In [6]:
dict_orgtype_encoder = {}
for i in list(df["ORGANIZATION_TYPE"].unique()):
    for k,v in encoder["ORGANIZATION_TYPE"].items():
        if str(round(v, 6)) in str(round(i, 6)):
            dict_orgtype_encoder[k] = round(v, 6)

In [7]:
dict_orgtype_encoder

{'Transport: type 3': 0.00386,
 'Other': 0.234323,
 'Business Entity Type 3': 0.221104,
 'Self-employed': 0.124913,
 'Industry: type 9': 0.010952,
 'Medicine': 0.036399,
 'School': 0.028919,
 'Construction': 0.021856,
 'Industry: type 3': 0.01066,
 'Government': 0.033833,
 'Security': 0.010559,
 'Electricity': 0.003089,
 'Kindergarten': 0.022373,
 'Industry: type 2': 0.001489,
 'Transport: type 4': 0.017554,
 'Business Entity Type 1': 0.019459,
 'Business Entity Type 2': 0.034317,
 'Transport: type 2': 0.007167,
 'Housing': 0.009619,
 'Police': 0.007613,
 'Services': 0.005122,
 'Security Ministries': 0.006419,
 'Industry: type 11': 0.008793,
 'Trade: type 7': 0.025466,
 'Trade: type 2': 0.006179,
 'Hotel': 0.003141,
 'Postal': 0.007014,
 'Military': 0.008566,
 'Trade: type 1': 0.001132,
 'Industry: type 5': 0.001948,
 'Industry: type 4': 0.002852,
 'Industry: type 12': 0.0012,
 'Agriculture': 0.00798,
 'University': 0.004315,
 'Bank': 0.008153,
 'Trade: type 6': 0.002052,
 'Industry: t

In [8]:
x = list(dict_orgtype_encoder.keys())
joblib.dump(x, "save_files/list_orgtype.joblib")

['save_files/list_orgtype.joblib']

In [9]:
# 1. Organization Type
dict_orgtype_encoder = {}
for k,v in encoder["ORGANIZATION_TYPE"].items():
    dict_orgtype_encoder[k.lower()] = round(v, 6)

In [10]:
dict_orgtype_encoder

{'other': 0.234323,
 'business entity type 3': 0.221104,
 'self-employed': 0.124913,
 'medicine': 0.036399,
 'business entity type 2': 0.034317,
 'government': 0.033833,
 'school': 0.028919,
 'trade: type 7': 0.025466,
 'kindergarten': 0.022373,
 'construction': 0.021856,
 'business entity type 1': 0.019459,
 'transport: type 4': 0.017554,
 'trade: type 3': 0.011356,
 'industry: type 9': 0.010952,
 'industry: type 3': 0.01066,
 'security': 0.010559,
 'housing': 0.009619,
 'industry: type 11': 0.008793,
 'military': 0.008566,
 'bank': 0.008153,
 'agriculture': 0.00798,
 'police': 0.007613,
 'transport: type 2': 0.007167,
 'postal': 0.007014,
 'security ministries': 0.006419,
 'trade: type 2': 0.006179,
 'restaurant': 0.005889,
 'services': 0.005122,
 'university': 0.004315,
 'industry: type 7': 0.00425,
 'transport: type 3': 0.00386,
 'industry: type 1': 0.003379,
 'hotel': 0.003141,
 'electricity': 0.003089,
 'industry: type 4': 0.002852,
 'trade: type 6': 0.002052,
 'industry: type 5'

In [11]:
org_type_encode = None
for k,v in dict_orgtype_encoder.items():
    if org_type.lower() == k:
        org_type_encode = v

In [12]:
org_type_encode

0.221104

In [13]:
# 2. Day
dict_day_encoder = {}
for k,v in encoder["WEEKDAY_APPR_PROCESS_START"].items():
    dict_day_encoder[k.lower()] = round(v, 6)

In [14]:
dict_day_encoder

{'tuesday': 0.175282,
 'wednesday': 0.168885,
 'monday': 0.164918,
 'thursday': 0.164518,
 'friday': 0.163695,
 'saturday': 0.110084,
 'sunday': 0.052619}

In [15]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [16]:
day_encode = None
for k,v in dict_day_encoder.items():
    if day.lower() == k:
        day_encode = v

In [17]:
day_encode

0.168885

### 2. Numerical Data (Standardization)

In [18]:
dict_stand

{'CNT_CHILDREN': 'save_files/joblib_standardization/CNT_CHILDREN.joblib',
 'AMT_INCOME_TOTAL': 'save_files/joblib_standardization/AMT_INCOME_TOTAL.joblib',
 'AMT_CREDIT': 'save_files/joblib_standardization/AMT_CREDIT.joblib',
 'AMT_ANNUITY': 'save_files/joblib_standardization/AMT_ANNUITY.joblib',
 'AMT_GOODS_PRICE': 'save_files/joblib_standardization/AMT_GOODS_PRICE.joblib',
 'REGION_POPULATION_RELATIVE': 'save_files/joblib_standardization/REGION_POPULATION_RELATIVE.joblib',
 'DAYS_BIRTH': 'save_files/joblib_standardization/DAYS_BIRTH.joblib',
 'DAYS_EMPLOYED': 'save_files/joblib_standardization/DAYS_EMPLOYED.joblib',
 'DAYS_REGISTRATION': 'save_files/joblib_standardization/DAYS_REGISTRATION.joblib',
 'DAYS_ID_PUBLISH': 'save_files/joblib_standardization/DAYS_ID_PUBLISH.joblib',
 'CNT_FAM_MEMBERS': 'save_files/joblib_standardization/CNT_FAM_MEMBERS.joblib',
 'HOUR_APPR_PROCESS_START': 'save_files/joblib_standardization/HOUR_APPR_PROCESS_START.joblib',
 'EXT_SOURCE_2': 'save_files/jobli

In [19]:
selected_feat = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
                 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'HOUR_APPR_PROCESS_START',
                 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'YEARS_BEGINEXPLUATATION_MODE',
                 'TOTALAREA_MODE', 'DAYS_LAST_PHONE_CHANGE',
                 'AMT_REQ_CREDIT_BUREAU_YEAR', 'WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE']

In [20]:
selected_feat[:-2] # Numerical variables

['AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'HOUR_APPR_PROCESS_START',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'YEARS_BEGINEXPLUATATION_MODE',
 'TOTALAREA_MODE',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_REQ_CREDIT_BUREAU_YEAR']

In [21]:
num_value = []

for i in selected_feat:
    if i == 'AMT_INCOME_TOTAL':
        applicant_income1 = joblib.load(dict_stand[i]).transform([[applicant_income]])
        num_value.append(applicant_income1)
    if i == 'AMT_CREDIT':
        credit_amount1 = joblib.load(dict_stand[i]).transform([[credit_amount]])
        num_value.append(credit_amount1)
    if i == 'AMT_ANNUITY':
        loan_annuity1 = joblib.load(dict_stand[i]).transform([[loan_annuity]])
        num_value.append(loan_annuity1)
    if i == 'AMT_GOODS_PRICE':
        goods_price1 = joblib.load(dict_stand[i]).transform([[goods_price]])
        num_value.append(goods_price1)
    if i == 'REGION_POPULATION_RELATIVE':
        region_pop1 = joblib.load(dict_stand[i]).transform([[region_pop]])
        num_value.append(region_pop1)
    if i == 'DAYS_BIRTH':
        applicant_age1 = joblib.load(dict_stand[i]).transform([[applicant_age]])
        num_value.append(applicant_age1)
    if i == 'DAYS_EMPLOYED':
        days_employed1 = joblib.load(dict_stand[i]).transform([[days_employed]])
        num_value.append(days_employed1)
    if i == 'DAYS_REGISTRATION':
        days_reg1 = joblib.load(dict_stand[i]).transform([[days_reg]])
        num_value.append(days_reg1)
    if i == 'DAYS_ID_PUBLISH':
        days_id1 = joblib.load(dict_stand[i]).transform([[days_id]])
        num_value.append(days_id1)
    if i == 'HOUR_APPR_PROCESS_START':
        hour1 = joblib.load(dict_stand[i]).transform([[hour]])
        num_value.append(hour1)
    if i == 'EXT_SOURCE_2':
        source_21 = joblib.load(dict_stand[i]).transform([[source_2]])
        num_value.append(source_21)
    if i == 'EXT_SOURCE_3':
        source_31 = joblib.load(dict_stand[i]).transform([[source_3]])
        num_value.append(source_31)
    if i == 'YEARS_BEGINEXPLUATATION_MODE':
        exp_mode1 = joblib.load(dict_stand[i]).transform([[exp_mode]])
        num_value.append(exp_mode1)
    if i == 'TOTALAREA_MODE':
        area_mode1 = joblib.load(dict_stand[i]).transform([[area_mode]])
        num_value.append(area_mode1)
    if i == 'DAYS_LAST_PHONE_CHANGE':
        days_phone1 = joblib.load(dict_stand[i]).transform([[days_phone]])
        num_value.append(days_phone1)
    if i == 'AMT_REQ_CREDIT_BUREAU_YEAR':
        credit_bureau1 = joblib.load(dict_stand[i]).transform([[credit_bureau]])
        num_value.append(credit_bureau1)

/Users/wanda/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/wanda/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/wanda/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/wanda/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/wanda/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/wanda/anaconda3/lib/python3.11/site-packages/skl

In [22]:
num_value

[array([[0.06620835]]),
 array([[-0.46004406]]),
 array([[-0.1582816]]),
 array([[-0.47947617]]),
 array([[-0.09657595]]),
 array([[1.39315836]]),
 array([[-0.41353008]]),
 array([[0.3230257]]),
 array([[0.49641994]]),
 array([[-0.5966161]]),
 array([[-0.98465467]]),
 array([[-1.76801616]]),
 array([[-0.09550409]]),
 array([[-1.16890495]]),
 array([[-0.30424599]]),
 array([[-0.54461827]])]

### 3. Predict Data

In [23]:
# Combine into DataFrame 

# Target = 1
df_pred1 = pd.DataFrame(np.concatenate(num_value), index = selected_feat[:-2]).T
df_pred1[selected_feat[-2]] = day_encode
df_pred1[selected_feat[-1]] = org_type_encode

In [24]:
df_pred1

,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,HOUR_APPR_PROCESS_START,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_MODE,TOTALAREA_MODE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_YEAR,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE
0,0.066208,-0.460044,-0.158282,-0.479476,-0.096576,1.393158,-0.41353,0.323026,0.49642,-0.596616,-0.984655,-1.768016,-0.095504,-1.168905,-0.304246,-0.544618,0.168885,0.221104


In [26]:
# Predict the Data

pred = int(model_logistic.predict(df_pred1))

msg1 = None
msg2 = None

if pred == 1:
    msg1 = "The Applicant Has Payment Difficulties"
    msg2 = "Loan Rejected"
elif pred == 0:
    msg1 = "The Applicant Doesn’t Have Payment Difficulties"
    msg2 = "Loan Accepted"

In [27]:
print(msg1)
print(msg2)

The Applicant Has Payment Difficulties
Loan Rejected
